In [1]:
import numpy as np
import tensorflow as tf
from load_dataset_VGG16 import load_dataset
import matplotlib.pyplot as plt

In [2]:
x_train,t_train = load_dataset('./train_dataset_VGG16',convert_type='RGB',flatten=True,normalize=True,one_hot_label=True)
x_test,t_test = load_dataset('./test_dataset_VGG16',convert_type='RGB',flatten=True,normalize=True,one_hot_label=True)

Load label : Done!
Load img : Done!
Load label : Done!
Load img : Done!


In [3]:
def list_shuffle(datas,labels):
    index_list = np.arange(0,datas.shape[0])
    np.random.shuffle(index_list)
    x_data = datas[index_list]
    t_data = labels[index_list]
    return x_data,t_data

In [4]:
x_train_shuffle,t_train_shuffle = list_shuffle(x_train,t_train)
x_test_shuffle,t_test_shuffle = list_shuffle(x_test,t_test)

In [5]:
t = tf.placeholder(tf.float32, shape=(None,2))
X = tf.placeholder(tf.float32, shape=(None,224*224*3))
# 入力層
input_layer = tf.reshape(X, [-1,224,224,3])

In [6]:
conv1_1 = tf.layers.conv2d(inputs=input_layer,filters=32,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
conv1_2 = tf.layers.conv2d(inputs=conv1_1,filters=32,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1_2,pool_size=(2,2),strides=(2,2),padding='same')

In [7]:
conv2_1 = tf.layers.conv2d(inputs=pool1,filters=64,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
conv2_2 = tf.layers.conv2d(inputs=conv2_1,filters=64,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2_2,pool_size=(2,2),strides=(2,2),padding='same')

In [8]:
conv3_1 = tf.layers.conv2d(inputs=pool2,filters=128,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
conv3_2 = tf.layers.conv2d(inputs=conv3_1,filters=128,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
conv3_3 = tf.layers.conv2d(inputs=conv3_2,filters=128,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
pool3 = tf.layers.max_pooling2d(inputs=conv3_3,pool_size=(2,2),strides=(2,2),padding='same')

In [9]:
conv4_1 = tf.layers.conv2d(inputs=pool3,filters=256,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
conv4_2 = tf.layers.conv2d(inputs=conv4_1,filters=256,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
conv4_3 = tf.layers.conv2d(inputs=conv4_2,filters=256,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
pool4 = tf.layers.max_pooling2d(inputs=conv4_3,pool_size=(2,2),strides=(2,2),padding='same')

In [10]:
conv5_1 = tf.layers.conv2d(inputs=pool4,filters=512,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
conv5_2 = tf.layers.conv2d(inputs=conv5_1,filters=512,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
conv5_3 = tf.layers.conv2d(inputs=conv5_2,filters=512,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
pool5 = tf.layers.max_pooling2d(inputs=conv5_3,pool_size=(2,2),strides=(2,2),padding='same')

In [11]:
conv6_1 = tf.layers.conv2d(inputs=pool5,filters=1024,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
pool6 = tf.layers.max_pooling2d(inputs=conv6_1,pool_size=(2,2),strides=(2,2),padding='same')

In [13]:
stddev = np.sqrt(2.0 / 4*4*1024)
h_W_fc1 = tf.Variable(tf.truncated_normal([4*4*1024,4096], stddev=stddev))
h_b_fc1 = tf.Variable(tf.constant(0.1, shape=[4096]))
pool5_flat = tf.reshape(pool5, [-1, 4*4*1024])
h_fc1 = tf.nn.relu(tf.matmul(pool5_flat, h_W_fc1) + h_b_fc1)

In [14]:
stddev = np.sqrt(2.0 / 4096)
h_W_fc2 = tf.Variable(tf.truncated_normal([4096,4096], stddev=stddev))
h_b_fc2 = tf.Variable(tf.constant(0.1, shape=[4096]))
h_fc2 = tf.nn.relu(tf.matmul(h_fc1, h_W_fc2) + h_b_fc2)

In [15]:
stddev = np.sqrt(2.0 / 4096)
W_fc3 = tf.Variable(tf.truncated_normal([4096,2], stddev=stddev))
b_fc3 = tf.Variable(tf.constant(0.1, shape=[2]))
fc = tf.nn.relu(tf.matmul(h_fc2, W_fc3) + b_fc3)

In [16]:
keep_prob = tf.placeholder(tf.float32) # ドロップアウトする割合
fc3_drop = tf.nn.dropout(fc, keep_prob)
y_conv = tf.nn.softmax(fc3_drop)

In [17]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=t))
loss = cross_entropy

optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(t,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
saver = tf.train.Saver()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

batch_size = 4

for i in range(1000):
    batch_mask = np.random.choice(x_train_shuffle.shape[0], batch_size)
    x_batch = x_train_shuffle[batch_mask]
    t_batch = t_train_shuffle[batch_mask]
    sess.run(train_step,feed_dict={X: x_batch, t: t_batch,keep_prob:0.5})
    if i % 100 == 0:
        train_acc, train_loss = sess.run([accuracy,loss], feed_dict={X: x_batch, t: t_batch,keep_prob:1.0})
        test_acc = sess.run(accuracy, feed_dict={X: x_test_shuffle, t: t_test_shuffle,keep_prob:1.0})
        print "[Train] step: %d, loss: %f, acc: %f, [Test] acc : %f" % (i, train_loss, train_acc,test_acc)

saver.save(sess, "./RoadSign-vgg-ckpt/roadsign-vgg")